<a href="https://colab.research.google.com/github/prachigupta2006/feature-engineering/blob/main/Binarization_on_titanic_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

In [39]:
from google.colab import files
uploaded = files.upload()

Saving Titanic-Dataset.csv to Titanic-Dataset (1).csv


In [40]:
df= pd.read_csv('Titanic-Dataset.csv')[['Age','Fare','Survived','Parch','SibSp']]

In [41]:
df.dropna(inplace=True)

In [42]:
df.head()

,Age,Fare,Survived,Parch,SibSp
0,22.0,7.2500,0,0,1
1,38.0,71.2833,1,0,1
2,26.0,7.9250,1,0,0
3,35.0,53.1000,1,0,1
4,35.0,8.0500,0,0,0


In [43]:
df['family']=df['SibSp']+df['Parch']

In [44]:
df.head()

,Age,Fare,Survived,Parch,SibSp,family
0,22.0,7.2500,0,0,1,1
1,38.0,71.2833,1,0,1,1
2,26.0,7.9250,1,0,0,0
3,35.0,53.1000,1,0,1,1
4,35.0,8.0500,0,0,0,0


In [45]:
df.drop(columns=['SibSp','Parch'],inplace=True)

In [46]:
df.head()

,Age,Fare,Survived,family
0,22.0,7.2500,0,1
1,38.0,71.2833,1,1
2,26.0,7.9250,1,0
3,35.0,53.1000,1,1
4,35.0,8.0500,0,0


# TRAIN TEST SPLIT

In [47]:
x=df.drop(columns=['Survived'])
y=df['Survived']

In [48]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [49]:
X_train.head()

,Age,Fare,family
328,31.0,20.5250,2
73,26.0,14.4542,1
253,30.0,16.1000,1
719,33.0,7.7750,0
666,25.0,13.0000,0


In [50]:
y_train.head()

,Survived
328,1
73,0
253,0
719,0
666,0


# **Without binarization**

# ACCURACY AND PREDICTION

In [51]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.6223776223776224

# CROSS VALIDATION

In [52]:
np.mean(cross_val_score(DecisionTreeClassifier(),X_train,y_train,cv=10,scoring='accuracy'))

0.6498487598306111

# **APPLYING BINARIZATION**

In [53]:
from sklearn.preprocessing import Binarizer

In [54]:
trf = ColumnTransformer([
    ('bin',Binarizer(copy=False),['family'])
],remainder='passthrough')

# Fitting and transformation

In [55]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)


# Creation of dataframe with binarized valued

In [56]:
pd.DataFrame(X_train_trf,columns=['family','Age','Fare'])

,family,Age,Fare
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


# ACCURACY AND PREDICTION after binarization


In [57]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)
y_pred2 = clf.predict(X_test_trf)
accuracy_score(y_test,y_pred2)

0.6153846153846154

#  CROSS VALIDATION after binarization

In [58]:
X_trf = trf.fit_transform(x)
np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,y,cv=10,scoring='accuracy'))

0.6262128325508607